## Check Questions

**Вопрос 1**: Что такое backpropagation?

Backpropagation -- этом метод подсчета градиента функции потерь по весам модели с помощью chain rule. 
Используется в нейросетях: каждый нейрон считает свои частные производные в forward pass-e, а потом они в обратном порядке перемножаются, чтобы получить итоговый градиент (так называемый backward pass).


**Вопрос 2**: Какие активационные функции вы знаете?

tanh(x), $\sigma(x) = \frac{1}{1 + \exp^{-x}}$, $rectifier(x) = \max(0, x)$

**Вопрос 3**: Чем они интересны, почему среди множества всех возможных функций были выбраны именно эти?

Эти функции нелинейные, поэтому можно аппроксимировать сложные зависимости, дифференцируемые (для backpropagation). Tanh и sigmoid выдают значения из [-1, 1] и [0, 1] соответственно, тем самым имитируя поведение нейронов в мозге. У tanh и sigmoid есть проблема -  vanishing gradients в очень глубоких сетях, т.к. очень маленькие производные, когда значения приближаются к 1.У ReLu нет такой проблемы.

**Вопрос 4**: Чем deep learning отличается от обычных нейронных сетей?

В deep neural networks много скрытых слоев, а в обычных нейронках 2-3 слоя.

**Вопрос 5**: Что такое свертка?

В применении к изображениям, это функция от какой-то части изображения, умножающая значения пикселей на веса и суммирующая результаты.
**Вопрос 6**: Что такое скрытое состояние?

Вектор, который хранится в RNN-cell и зависит от текущего входа и предыдущего своего значения. Таким образом RNN может учитывать предыдущий контекст.

**Вопрос 7**: Что такое embedding?

<Ответ>

In [1]:
import tensorflow as tf
from tensorflow.python.ops import rnn_cell
from scipy.ndimage.interpolation import shift
import numpy as np

In [2]:
print(tf.__version__)

0.11.0rc0


### Utilities

Now that we have our training corpus for our language model (optionally you could gather an actual corpus from the web :), we can now create our first utility, `Vocab`, that will hold the mapping from words to an index, and perfom the conversions from words to indices and vice-versa:

In [3]:
# Utilities:
class Vocab(object):
    __slots__ = ["word2index", "index2word", "unknown"]
    
    def __init__(self, index2word = None):
        self.word2index = {}
        self.index2word = []
        
        # add unknown word:
        self.add_words(["**UNKNOWN**"])
        self.unknown = 0
        
        if index2word is not None:
            self.add_words(index2word)
                
    def add_words(self, words):
        for word in words:
            if word not in self.word2index:
                self.word2index[word] = len(self.word2index)
                self.index2word.append(word)
                       
    def __call__(self, line):
        """
        Convert from numerical representation to words
        and vice-versa.
        """
        if type(line) is np.ndarray:
            return " ".join([self.index2word[word] for word in line])
        if type(line) is list:
            if len(line) > 0:
                if line[0] is int:
                    return " ".join([self.index2word[word] for word in line])
            indices = np.zeros(len(line), dtype=np.int32)
        else:
            line = line.split(" ")
            indices = np.zeros(len(line), dtype=np.int32)
        
        for i, word in enumerate(line):
            indices[i] = self.word2index.get(word, self.unknown)
            
        return indices
    
    @property
    def size(self):
        return len(self.index2word)
    
    def __len__(self):
        return len(self.index2word)

### Create a Mapping from numbers to words

Now we can use the `Vocab` class to gather all the words and store an Index:

In [39]:
lines = ["asd qwe qwe", "quick brown fox jumps over the lazy dog", "dog asd fox lazy", "qwe asd asd fox jumps",
         " qwerty asd qwe qwe asd asd over", "dog lazy asd qwe dcp"]
vocab = Vocab()
for line in lines:
    vocab.add_words(line.split(" "))

To send our sentences in one big chunk to our neural network we transform each sentence into a row vector and place each of these rows into a bigger matrix that holds all these rows. Not all sentences have the same length, so we will pad those that are too short with 0s in `pad_into_matrix`:

In [40]:
def pad_into_matrix(rows, padding = 0):
    if len(rows) == 0:
        return np.array([0, 0], dtype=np.int32)
    lengths = list(map(len, rows))
    width = max(lengths)
    height = len(rows)
    mat = np.empty([height, width], dtype=rows[0].dtype)
    mat.fill(padding)
    for i, row in enumerate(rows):
        mat[i, 0:len(row)] = row
    return mat, lengths

In [41]:
# transform into big numerical matrix of sentences:
numerical_lines = []
for line in lines:
    numerical_lines.append(vocab(line))
numerical_lines, numerical_lengths = pad_into_matrix(numerical_lines)

In [42]:
numerical_lengths

[3, 8, 4, 5, 8, 5]

In [50]:
def get_batches(numerical_lines, numerical_lengths, batch_size):
    current_batch_num = 0
    train_size = len(numerical_lines)
    while current_batch_num * batch_size < train_size:
        start = current_batch_num * batch_size
        end = (current_batch_num + 1) * batch_size
        yield numerical_lines[start:end], numerical_lengths[start:end], shift(numerical_lines[start:end], shift=(0, -1))
        current_batch_num += 1

In [51]:
hidden_size = 200
num_layers = 3
batch_size = 3
num_items = len(vocab)
num_epochs = 200
width = numerical_lines.shape[1]

In [58]:
graph = tf.Graph()
with graph.as_default():
    inputs = tf.placeholder(tf.int64, shape=(batch_size, width), name='inputs')
    labels = tf.placeholder(tf.int64, shape=(batch_size, width), name='labels')
    lengths = tf.placeholder(tf.int64, shape=(batch_size), name='lengths')
    embeddings = tf.Variable(tf.random_uniform([num_items, hidden_size], -1.0, 1.0))
    # weights and bias for fully connected layer before softmax
    W = tf.Variable(tf.truncated_normal([hidden_size, num_items], stddev=np.sqrt(2.0 / hidden_size)))
    b = tf.Variable(tf.zeros([]))
    cell = rnn_cell.LSTMCell(hidden_size)
    rnn_multicell = rnn_cell.MultiRNNCell([cell] * num_layers)
    embedding = tf.nn.embedding_lookup(embeddings, inputs)
    outputs, _ = tf.nn.dynamic_rnn(rnn_multicell, embedding, lengths, dtype=tf.float32)
    print(outputs.get_shape())
    outputs = tf.reshape(outputs, [-1, hidden_size])
    logits = tf.matmul(outputs, W) + b
    logits = tf.reshape(logits, [batch_size, width, num_items])
    print(logits.get_shape())
    loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits, labels) *
                          tf.to_float(tf.not_equal(labels, 0)))
    optimizer = tf.train.AdamOptimizer().minimize(loss)

(3, 8, 200)
(3, 8, 14)


In [59]:
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    for epoch in range(num_epochs):
        batch_number = 0
        print("Starting epoch {}...".format(epoch))
        for (inputs_batch, lengths_batch, labels_batch) in get_batches(numerical_lines, numerical_lengths, batch_size):
            print("inputs: {}, labels: {}".format(inputs_batch, labels_batch))
            feedDict = {inputs: inputs_batch, lengths: lengths_batch, labels: labels_batch}
            print("\tStarting batch {}...".format(batch_number))
            batch_cost, _ = session.run([loss, optimizer], feed_dict=feedDict)
            print("\tCost: {}".format(batch_cost))
            batch_number += 1

Starting epoch 0...
inputs: [[ 1  2  2  0  0  0  0  0]
 [ 3  4  5  6  7  8  9 10]
 [10  1  5  9  0  0  0  0]], labels: [[ 2  2  0  0  0  0  0  0]
 [ 4  5  6  7  8  9 10  0]
 [ 1  5  9  0  0  0  0  0]]
	Starting batch 0...
	Cost: 1.317484736442566
inputs: [[ 2  1  1  5  6  0  0  0]
 [11 12  1  2  2  1  1  7]
 [10  9  1  2 13  0  0  0]], labels: [[ 1  1  5  6  0  0  0  0]
 [12  1  2  2  1  1  7  0]
 [ 9  1  2 13  0  0  0  0]]
	Starting batch 1...
	Cost: 1.6511636972427368
Starting epoch 1...
inputs: [[ 1  2  2  0  0  0  0  0]
 [ 3  4  5  6  7  8  9 10]
 [10  1  5  9  0  0  0  0]], labels: [[ 2  2  0  0  0  0  0  0]
 [ 4  5  6  7  8  9 10  0]
 [ 1  5  9  0  0  0  0  0]]
	Starting batch 0...
	Cost: 1.2729976177215576
inputs: [[ 2  1  1  5  6  0  0  0]
 [11 12  1  2  2  1  1  7]
 [10  9  1  2 13  0  0  0]], labels: [[ 1  1  5  6  0  0  0  0]
 [12  1  2  2  1  1  7  0]
 [ 9  1  2 13  0  0  0  0]]
	Starting batch 1...
	Cost: 1.6022876501083374
Starting epoch 2...
inputs: [[ 1  2  2  0  0  0  

In [ ]:
# class RnnLm(object):
#     def __init__(num_layers=3, batch_size=3, hidden_size=200, method='adadelta'):
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.batch_size = batch_size
#         self.method = method